In [31]:
import numpy
import matplotlib
import pandas as pd
import csv

In [32]:
DATA_PATH = './data/twitter-datasets/'
TRAIN_POS = DATA_PATH + 'train_pos.txt'
TRAIN_NEG = DATA_PATH + 'train_neg.txt'

In [33]:
TRAIN_POS_CSV = DATA_PATH + 'train_pos.csv'
TRAIN_NEG_CSV = DATA_PATH + 'train_neg.csv'

In [34]:
def textToCsv(text_path, csv_path):
    with open(text_path, 'r') as in_file:
        stripped = (line.strip() for line in in_file)
        lines = (line.split(",") for line in stripped if line)
        with open(csv_path, 'w') as out_file:
            writer = csv.writer(out_file)
            writer.writerows(lines)

In [35]:
textToCsv(TRAIN_POS, TRAIN_POS_CSV)
textToCsv(TRAIN_NEG, TRAIN_NEG_CSV)

In [36]:
train_pos_df = pd.read_csv(TRAIN_POS_CSV, header=None, sep="\n")
train_neg_df = pd.read_csv(TRAIN_NEG_CSV, header=None, sep="\n")

In [37]:
train_pos_df.shape

(100000, 1)

In [38]:
train_neg_df.shape

(100000, 1)

In [39]:
train_pos_df.head(10)

,0
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,""" <user> just put casper in a box ! "" looved t..."
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...
5,<user> yay ! ! #lifecompleted . tweet / facebo...
6,<user> #1dnextalbumtitle : feel for you / roll...
7,workin hard or hardly workin rt <user> at hard...
8,<user> i saw . i'll be replying in a bit .
9,this is were i belong


In [40]:
train_neg_df.head(10)

,0
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...
5,wish i could be out all night tonight ! <user>
6,<user> i got kicked out the wgm
7,rt <user> <user> <user> yes she is ! u tell it...
8,why is she so perfect <url>
9,<user> hi harry ! did u havea good time in aus...


In [48]:
train_df = pd.concat((train_pos_df, train_neg_df)).sample(frac=1).reset_index(drop=True)

In [49]:
train_df

,0
0,don't think it's fair that us & canadian direc...
1,if i help free <user> do i still have to pay d...
2,<user> wish i was in bristol xx
3,<user> i have too much of the dark side in me ...
4,rt <user> you know you've found love when you ...
...,...
199995,fats burned lots today ! good night yoroobun !
199996,"new blood ( mass market paperback in 1636 , th..."
199997,dat guy tryin 2 form familiarity wif gbemi is ...
199998,"i feel sick and dizzy , time for bed"


In [50]:
TRAIN_NEG.replace('.txt', '.csv')

'./data/twitter-datasets/train_neg.csv'

In [ ]:
def loadDataSet(pos_path, neg_path):
    textToCsv(pos_path, pos_path.replace('.txt', '.csv'))

In [51]:
def loadDataSet(path):
    new_path = path.replace('.txt', '.csv')
    textToCsv(path, new_path)
    return pd.read_csv(new_path, header=None, sep="\n")

In [54]:
def loadFullDataSet(path_pos):
    path_neg = path_pos.replace('_pos', '_neg')
    data_pos = loadDataSet(path_pos)
    data_neg = loadDataSet(path_neg)
    
    data_pos['mood'] = 1
    data_neg['mood'] = 0
    
    return pd.concat((data_pos, data_neg)).sample(frac=1).reset_index(drop=True)

In [55]:
FULL_POS_PATH = DATA_PATH + 'train_pos_full.txt'
full_data = loadFullDataSet(FULL_POS_PATH)

In [59]:
full_data = full_data.rename(columns={0:"tweet"})

### Let's start installing all the features we need for bert.

In [64]:
from sklearn.model_selection import train_test_split

First we need to prepare our dataset in order to match with the format of BERT.

In [61]:
train_df_bert = pd.DataFrame({
    'id':range(len(full_data)),
    'label':full_data['mood'],
    'alpha':['a']*full_data.shape[0],
    'text': full_data['tweet']
})

train_df_bert.head()

,id,label,alpha,text
0,0,0,a,i said it straight up . you said you dont love...
1,1,1,a,<user> gud lad yeah i bet they did buds ! u wa...
2,2,1,a,rt <user> goodmorning <user> <3 i love you gir...
3,3,0,a,<user> and they are no additional date
4,4,1,a,"<user> liam , thank you for doing this ! we lo..."


In [62]:
FINAL_TRAIN = DATA_PATH + 'train.tsv'


train_df_bert.to_csv(FINAL_TRAIN, sep='\t', index=False, header=False)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
little_train_df_bert = train_df_bert. 

In [63]:
from __future__ import absolute_import, division, print_function

import csv
import os
import sys
import logging

logger = logging.getLogger()
csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples